# torchtext ライブラリを使ったテキスト分類

ここでは、テキスト分類のためのデータセットを構築するために torchtext ライブラリを使用する方法を紹介します。  
ユーザは以下のような柔軟性を持つことができます。

   - イテレータとして生データにアクセスする
   - データ処理パイプラインを構築して、生のテキスト文字列を ``torch.Tensor`` に変換し、モデルの学習に使用する
   - [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) を使ってデータをシャッフルし、反復処理する


## 生のデータセットイテレータへのアクセス

torchtext ライブラリには、生のテキスト文字列を返す生のデータセットイテレータがいくつか用意されています。  
例えば、 ``AG_NEWS`` データセットイテレータは、ラベルとテキストのタプルとして生データを得ることができます。  

torchtextのデータセットにアクセスするには、https://github.com/pytorch/data の説明に従ってtorchdataをインストールしてください。


In [ ]:
!pip install -U portalocker>=2.0.0

In [2]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))

In [8]:
next(train_iter)

(3,
 'Stocks End Up, But Near Year Lows (Reuters) Reuters - Stocks ended slightly higher on Friday\\but stayed near lows for the year as oil prices surged past  #36;46\\a barrel, offsetting a positive outlook from computer maker\\Dell Inc. (DELL.O)')

## データ処理パイプラインの用意

ボキャブラリー、ワードベクター、トークナイザーなど、torchtextライブラリの非常に基本的な構成要素について再確認しました。  
これらは生のテキスト文字列に対する基本的なデータ処理のビルディングブロックです。

ここでは、トークナイザーとボキャブラリーを使った典型的な自然言語処理例を紹介します。  
まず、生の学習データセットから語彙を作る。ここでは、ビルトインの
ファクトリー関数 `build_vocab_from_iterator` を使う。この関数では、トークンのリストかイテレータを受け取ります。  
また、語彙に追加する特殊なシンボルを渡すこともできます。
語彙に追加する特別な記号を渡すこともできます。



In [10]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

vocabularyブロックは，トークンのリストを整数に変換できます。

    vocab(['here', 'is', 'an', 'example'])
    >>> [475, 21, 30, 5297]

トークナイザーとボキャブラリーでテキスト処理パイプラインを準備します。  
テキストパイプラインとラベルパイプラインは、データセットイテレータからの生データ文字列を処理するために使用します。


In [11]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

テキストパイプラインは，語彙に定義されたルックアップテーブルに基づいて，文字列を整数のリストに変換します。  
ラベルパイプラインは、ラベルを整数に変換します。  
例えば

    text_pipeline('here is the an example')
    >>> [475, 21, 2, 30, 5297]
    label_pipeline('10')
    >>> 9




## データバッチとイテレータを生成する

データローダー（DataLoader）は ``getitem()`` と ``len()`` プロトコルを実装したマップ形式のデータセットで動作し、インデックス/キーからデータサンプルへのマップを表現します。また、引数として ``False`` を指定した、反復処理可能なデータセットでも動作します。

モデルに送信する前に、 ``collate_fn`` 関数は ``DataLoader`` から生成されたサンプルのバッチを処理する。``collate_fn`` への入力は ``DataLoader`` にあるバッチサイズのデータであり、 ``collate_fn`` は先に宣言したデータ処理パイプラインに従ってそれらを処理する。ここで注意してほしいのは、 ``collate_fn`` がトップレベルの def として宣言されていることだ。 これにより、この関数が各ワーカーで利用できるようになる。

この例では、元のデータバッチの入力に含まれるテキストエントリーをリストにパックし、一つのテンソルとして連結して ``nn.EmbeddingBag`` の入力とします。offsetはテキストテンソル中の個々のシーケンスの開始インデックスを表すデリミタのテンソルである。ラベルは個々のテキストエントリのラベルを保存したテンソルである。

In [12]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

## モデルの定義

モデルは [nn.EmbeddingBag](https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag)レイヤーと、分類を目的とした線形レイヤーから構成されます。  
``nn.EmbeddingBag``はデフォルトのモードが "mean "で、埋め込みの "bag "の平均値を計算します。  
ここでは、テキストのエントリは異なる長さを持っていますが、テキストの長さはオフセットで保存されているので、nn.EmbeddingBagモジュールはここでパディングを必要としません。  
さらに、``nn.EmbeddingBag`` は、一連の処理を行う際のパフォーマンスとメモリ効率を向上させることができます。

In [13]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## インスタンスの起動

``AG_NEWS`` のデータセットには4つのラベルがあるので、クラスの数は4です。

   1 : 世界
   2 : スポーツ
   3 : ビジネス
   4 : 科学/技術

埋め込み次元を64としたモデルを構築します。  
vocab sizeはvocabulary instanceの長さと等しく、クラス数はラベルの数に等しくなります。

In [14]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

## モデルの学習と結果の評価のための関数の定義



In [15]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## データセットの分割とモデルの実行

元の ``AG_NEWS`` には検証用データセットがないため、学習用データセットを分割します。  
データセットを訓練用と検証用に分割し、分割比率を 0.95 (train) と0.05 (valid) とします。  
ここで使用するのは
[torch.utils.data.dataset.random_split](https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split)というPyTorchのコアライブラリにある関数を使います。

[CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss)を使用します。  
この基準は、 ``nn.LogSoftmax()`` と ``nn.NLLLoss()`` を1つのクラスに統合したものです。  
これは，C個のクラスからなる分類問題を学習するときに役立ちます。  
最適化器には[SGD](https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html)を使います。初期学習率は 5.0 に設定します。  
[ステップLR](https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR)は、エポックを通して学習率を調整するために使います。

In [16]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.715
| epoch   1 |  1000/ 1782 batches | accuracy    0.868
| epoch   1 |  1500/ 1782 batches | accuracy    0.884
-----------------------------------------------------------
| end of epoch   1 | time:  4.09s | valid accuracy    0.880 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.903
| epoch   2 |  1000/ 1782 batches | accuracy    0.907
| epoch   2 |  1500/ 1782 batches | accuracy    0.906
-----------------------------------------------------------
| end of epoch   2 | time:  4.17s | valid accuracy    0.903 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.919
| epoch   3 |  1000/ 1782 batches | accuracy    0.920
| epoch   3 |  1500/ 1782 batches | accuracy    0.916
-----------------------------------------------------------
| end of epoch   3 | time:  4.27s | valid accuracy    0.900 
-------------------------------

## テストデータセットでモデルを評価




テストデータセットの結果を確認します

In [17]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.907


## ランダムなニュースでテスト

学習済みモデルを使って、ゴルフのニュースをテストしてみましょう。




In [18]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
